## MA

In [1]:
import menpo.io as mio
import numpy as np
import tensorflow as tf
from sklearn import svm
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

%matplotlib inline

### data preparation

In [2]:
stock_slices = mio.import_pickle('/homes/yz4009/wd/gitdev/MarketAnalysor/DataAnalysis/data/090_030_slices.pkl')

In [3]:
len(stock_slices)

1490

In [4]:
data = []
for s in stock_slices:
    for k in s.keys():
        data += s[k]

In [5]:
def label_y(trend):
    cl = [0, 0, 1]
    if trend > 0.3:
        cl = [1, 0, 0]
    elif trend < -0.3:
        cl = [0, 1, 0]
        
    return cl



data_X = [d['data'] for d in data]
data_Y = [label_y(d['trend']) for d in data]

In [6]:
data_X = np.array(data_X)
data_Y = np.array(data_Y)

data_X[np.isnan(data_X)] = 0

In [7]:
n_data = len(data)
n_train = int(n_data * 0.8)
n_valid = int(n_data * 0.1)

In [8]:
train_X = data_X[:n_train]
train_Y = data_Y[:n_train]
valid_X = data_X[n_train:n_train+n_valid]
valid_Y = data_Y[n_train:n_train+n_valid]
test_X = data_X[n_train+n_valid:]
test_Y = data_Y[n_train+n_valid:]

### SVM Prediction

### Tensor Flow Prediction

In [9]:
train_X.shape

(2650423, 89)

In [10]:
np.isnan(train_Y).any()

False

In [11]:
train_Y.shape

(2650423, 3)

In [12]:
batch_size = 100

sess = tf.InteractiveSession()

x = tf.placeholder(tf.float32, [None, 89])
y_ = tf.placeholder(tf.float32, [None, 3])

W1 = tf.Variable(tf.random_uniform([89, 3], -1.0, 1.0))
b1 = tf.Variable(tf.random_uniform([3], -1.0, 1.0))

pred = tf.nn.softmax(tf.matmul(x, W1) + b1)

keep_prob = tf.placeholder(tf.float32)
y = tf.nn.dropout(pred, keep_prob)

cross_entropy = -tf.reduce_sum(y_*tf.log(tf.clip_by_value(y,1e-10,1.0)))
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


sess.run(tf.initialize_all_variables())

for i in xrange(train_X.shape[0] / batch_size):
    batch_xs = train_X[i * batch_size: i * batch_size + batch_size]
    batch_ys = train_Y[i * batch_size: i * batch_size + batch_size]
    
    if i % batch_size == 0:
        train_accuracy = accuracy.eval(feed_dict={
        x:batch_xs, y_: batch_ys, keep_prob: 1.0})
        print("step %d, training accuracy %g"%(i, train_accuracy))

    train_step.run(feed_dict={x: batch_xs, y_: batch_ys, keep_prob: 0.5})



print(sess.run(accuracy, feed_dict={x: test_X, y_: test_Y, keep_prob: 1.0}))



step 0, training accuracy 0.46
step 100, training accuracy 0.95
step 200, training accuracy 1
step 300, training accuracy 1
step 400, training accuracy 1
step 500, training accuracy 0.98
step 600, training accuracy 1
step 700, training accuracy 1
step 800, training accuracy 1
step 900, training accuracy 1
step 1000, training accuracy 1
step 1100, training accuracy 1
step 1200, training accuracy 0.94
step 1300, training accuracy 1
step 1400, training accuracy 1
step 1500, training accuracy 1
step 1600, training accuracy 1
step 1700, training accuracy 0.42
step 1800, training accuracy 1
step 1900, training accuracy 1
step 2000, training accuracy 0.43
step 2100, training accuracy 1
step 2200, training accuracy 1
step 2300, training accuracy 1
step 2400, training accuracy 1
step 2500, training accuracy 1
step 2600, training accuracy 1
step 2700, training accuracy 1
step 2800, training accuracy 1
step 2900, training accuracy 1
step 3000, training accuracy 0.99
step 3100, training accuracy 1

In [13]:
ret = y.eval(feed_dict={x: test_X})

InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder_2' with dtype float
	 [[Node: Placeholder_2 = Placeholder[dtype=DT_FLOAT, shape=[], _device="/job:localhost/replica:0/task:0/cpu:0"]()]]
Caused by op u'Placeholder_2', defined at:
  File "/vol/atlas/homes/yz4009/miniconda/envs/gitdev/lib/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/vol/atlas/homes/yz4009/miniconda/envs/gitdev/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/vol/atlas/homes/yz4009/miniconda/envs/gitdev/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/vol/atlas/homes/yz4009/miniconda/envs/gitdev/lib/python2.7/site-packages/traitlets/config/application.py", line 596, in launch_instance
    app.start()
  File "/vol/atlas/homes/yz4009/miniconda/envs/gitdev/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 442, in start
    ioloop.IOL

In [ ]:
np.sum((ret[:,0] - ret[:,1]) < 0)

#### demo

In [11]:
import tensorflow as tf
import numpy as np

# Create 100 phony x, y data points in NumPy, y = x * 0.1 + 0.3
x_data = np.random.rand(100).astype(np.float32)
y_data = x_data * 0.1 + 0.3

# Try to find values for W and b that compute y_data = W * x_data + b
# (We know that W should be 0.1 and b 0.3, but Tensorflow will
# figure that out for us.)
W = tf.Variable(tf.zeros([1]))
b = tf.Variable(tf.zeros([1]))

y = W * x_data + b

# Minimize the mean squared errors.
loss = tf.reduce_mean(tf.square(y - y_data))
optimizer = tf.train.GradientDescentOptimizer(0.5)
train = optimizer.minimize(loss)

# Before starting, initialize the variables.  We will 'run' this first.
init = tf.initialize_all_variables()

# Launch the graph.
sess = tf.Session()
sess.run(init)

# Fit the line.
for step in xrange(401):
    sess.run(train)
    if step % 20 == 0:
        print(step, sess.run(W), sess.run(b))

# Learns best fit is W: [0.1], b: [0.3]

(0, array([ 0.19459628], dtype=float32), array([ 0.35310328], dtype=float32))
(20, array([ 0.11602166], dtype=float32), array([ 0.29100075], dtype=float32))
(40, array([ 0.10521366], dtype=float32), array([ 0.29707155], dtype=float32))
(60, array([ 0.10169658], dtype=float32), array([ 0.29904705], dtype=float32))
(80, array([ 0.1005521], dtype=float32), array([ 0.29968989], dtype=float32))
(100, array([ 0.10017967], dtype=float32), array([ 0.2998991], dtype=float32))
(120, array([ 0.10005846], dtype=float32), array([ 0.29996717], dtype=float32))
(140, array([ 0.10001902], dtype=float32), array([ 0.29998934], dtype=float32))
(160, array([ 0.10000618], dtype=float32), array([ 0.29999655], dtype=float32))
(180, array([ 0.10000201], dtype=float32), array([ 0.29999888], dtype=float32))
(200, array([ 0.10000065], dtype=float32), array([ 0.29999965], dtype=float32))
(220, array([ 0.10000022], dtype=float32), array([ 0.29999989], dtype=float32))
(240, array([ 0.1000001], dtype=float32), array(

#### mnist

In [34]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [66]:
sess = tf.InteractiveSession()

x = tf.placeholder(tf.float32, [None, 784])

W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

y = tf.nn.softmax(tf.matmul(x, W) + b)

y_ = tf.placeholder(tf.float32, [None, 10])

cross_entropy = -tf.reduce_sum(y_*tf.log(y))

train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

init = tf.initialize_all_variables()

sess.run(init)

for i in range(1000):
  batch_xs, batch_ys = mnist.train.next_batch(100)
  train_step.run(feed_dict={x: batch_xs, y_: batch_ys})

correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))

accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

Exception AssertionError: AssertionError() in <bound method InteractiveSession.__del__ of <tensorflow.python.client.session.InteractiveSession object at 0x7f9cdcf47dd0>> ignored


0.9168


#### deep mnist

In [24]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

In [25]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [26]:
sess = tf.InteractiveSession()

Exception AssertionError: AssertionError() in <bound method InteractiveSession.__del__ of <tensorflow.python.client.session.InteractiveSession object at 0x7f9f3aab0690>> ignored


In [27]:
x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None, 10])

In [28]:
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

In [29]:
x_image = tf.reshape(x, [-1,28,28,1])

In [30]:
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

In [31]:
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [32]:
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

In [33]:
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [34]:
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

In [35]:
cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.initialize_all_variables())
for i in range(20000):
  batch = mnist.train.next_batch(50)
  if i%100 == 0:
    train_accuracy = accuracy.eval(feed_dict={
        x:batch[0], y_: batch[1], keep_prob: 1.0})
    print("step %d, training accuracy %g"%(i, train_accuracy))
  train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

print("test accuracy %g"%accuracy.eval(feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

step 0, training accuracy 0.08
step 100, training accuracy 0.82
step 200, training accuracy 0.92
step 300, training accuracy 0.84
step 400, training accuracy 0.98
step 500, training accuracy 0.9
step 600, training accuracy 0.96
step 700, training accuracy 0.96
step 800, training accuracy 0.88
step 900, training accuracy 1
step 1000, training accuracy 0.96
step 1100, training accuracy 0.92
step 1200, training accuracy 0.94
step 1300, training accuracy 0.92
step 1400, training accuracy 0.94
step 1500, training accuracy 0.94
step 1600, training accuracy 1
step 1700, training accuracy 0.96
step 1800, training accuracy 0.98
step 1900, training accuracy 0.96
step 2000, training accuracy 0.96
step 2100, training accuracy 1
step 2200, training accuracy 0.98
step 2300, training accuracy 0.96
step 2400, training accuracy 0.96
step 2500, training accuracy 0.96
step 2600, training accuracy 0.96
step 2700, training accuracy 1
step 2800, training accuracy 1
step 2900, training accuracy 1
step 3000, 

In [52]:
result = y_conv.eval(feed_dict={
    x: mnist.test.images[:1], y_: range(10), keep_prob: 1.0})

In [49]:
result[0]

array([  1.39344040e-11,   6.08332940e-10,   3.15340254e-10,
         4.88835361e-10,   5.84304870e-12,   2.30223094e-13,
         8.29810597e-16,   1.00000000e+00,   4.27762253e-12,
         2.36218245e-09], dtype=float32)